In [1]:
!pip install snowflake-snowpark-python 



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from snowflake.snowpark import Session
import sys
import logging

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')




In [3]:
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ek87911.ap-southeast-1",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    return Session.builder.configs(connection_parameters).create()   

In [4]:
def main():
    session = get_snowpark_session()

    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(2)

    customer_df = session.sql("select c_custkey,c_name,c_phone,c_mktsegment from snowflake_sample_data.tpch_sf1.customer limit 10")
    customer_df.show(5)

if __name__ == '__main__':
    main()  

02:52:05 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.10.2, Platform: Windows-10-10.0.22621-SP0
02:52:05 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
02:52:05 - INFO - Setting use_openssl_only mode to False
02:52:09 - INFO - Snowpark Session information: 
"version" : 1.5.1,
"python.version" : 3.10.2,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1418670647832610,
"os.name" : Windows

02:52:09 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
02:52:09 - INFO - query execution done
02:52:09 - INFO - Number of results in first chunk: 0
02:52:09 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
02:52:09 - INFO - query execution done
02:52

In [5]:
!pip install snowflake-connector-Python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install snowflake-sqlalchemy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:

import os
def traverse_directory(directory,file_extension) -> list:
    local_file_path = []
    file_name = []  # List to store CSV file paths
    partition_dir = []
    print(directory)
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                file_name.append(file)
                partition_dir.append(root.replace(directory, ""))
                local_file_path.append(file_path)

    return file_name,partition_dir,local_file_path

def main():
    # Specify the directory path to traverse
    directory_path = "Users\LENOVO\Downloads\snowpark20"
    csv_file_name, csv_partition_dir , csv_local_file_path= traverse_directory(directory_path,'.csv')
   # parquet_file_name, parquet_partition_dir , parquet_local_file_path= traverse_directory(directory_path,'.parquet')
   # json_file_name, json_partition_dir , json_local_file_path= traverse_directory(directory_path,'.json')
    stage_location = 'SALES_DWH.SOURCE.MY_INTERNAL_STG'

    
    csv_index = 0
    for file_element in csv_file_name:
        print("hi")
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        csv_index+=1
'''
    parquet_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        parquet_local_file_path[parquet_index], 
                        stage_location+"/"+parquet_partition_dir[parquet_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        parquet_index+=1
    
    json_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        json_local_file_path[json_index], 
                        stage_location+"/"+json_partition_dir[json_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        json_index+=1  
    '''
    
if __name__ == '__main__':
    print("hi")
    main()

hi
Users\LENOVO\Downloads\snowpark20


In [8]:
!pip install snowflake-snowpark-python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ek87911.ap-southeast-1",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    return Session.builder.configs(connection_parameters).create()  

def ingest_in_sales(session)-> None:
    session.sql(" \
            copy into sales_dwh.source.in_sales_order from ( \
            select \
            sales_dwh.source.in_sales_order_seq.nextval, \
            t.$1::text as order_id, \
            t.$2::text as customer_name, \
            t.$3::text as mobile_key,\
            t.$4::number as order_quantity, \
            t.$5::number as unit_price, \
            t.$6::number as order_valaue,  \
            t.$7::text as promotion_code , \
            t.$8::number(10,2)  as final_order_amount,\
            t.$9::number(10,2) as tax_amount,\
            t.$10::date as order_dt,\
            t.$11::text as payment_status,\
            t.$12::text as shipping_status,\
            t.$13::text as payment_method,\
            t.$14::text as payment_provider,\
            t.$15::text as mobile,\
            t.$16::text as shipping_address,\
            metadata$filename as stg_file_name,\
            metadata$file_row_number as stg_row_numer,\
            metadata$file_last_modified as stg_last_modified\
            from \
            @sales_dwh.source.my_internal_stg \
            (                                                             \
                file_format => 'sales_dwh.common.my_csv_format'           \
            ) t  )  on_error = 'Continue'     \
            "
            ).collect()



def main():

    #get the session object and get dataframe
    session = get_snowpark_session()

    #ingest in sales data
    ingest_in_sales(session)   

if __name__ == '__main__':
    main()

02:52:25 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.10.2, Platform: Windows-10-10.0.22621-SP0
02:52:25 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
02:52:27 - INFO - Snowpark Session information: 
"version" : 1.5.1,
"python.version" : 3.10.2,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1418670647836714,
"os.name" : Windows

02:52:27 - INFO - query: [copy into sales_dwh.source.in_sales_order from (             select             ...]
02:52:28 - INFO - query execution done
02:52:29 - INFO - Number of results in first chunk: 1


In [10]:
import sys
import logging

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col,lit,row_number, rank
from snowflake.snowpark import Window

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')


def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ek87911.ap-southeast-1",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    return Session.builder.configs(connection_parameters).create()   

def filter_dataset(df, column_name, filter_criterian) -> DataFrame:
   
    return_df = df.filter(col(column_name) == filter_criterian)

    return return_df

def main():

   
    session = get_snowpark_session()
    sales_df = session.sql("select * from us_sales_order")

    paid_sales_df = filter_dataset(sales_df,'PAYMENT_STATUS','Paid')
    shipped_sales_df = filter_dataset(paid_sales_df,'SHIPPING_STATUS','Delivered')

    country_sales_df = shipped_sales_df.with_column('Country',lit('FR')).with_column('Region',lit('EU'))
    forex_df = session.sql("select * from sales_dwh.common.exchange_rate")
    sales_with_forext_df = country_sales_df.join(forex_df,country_sales_df['order_dt']==forex_df['echange_rate_dt'],join_type='outer')

    print(sales_with_forext_df.count())
    unique_orders = sales_with_forext_df.with_column('order_rank',rank().over(Window.partitionBy(col("order_dt")).order_by(col('_metadata_last_modified').desc()))).filter(col("order_rank")==1).select(col('SALES_ORDER_KEY').alias('unique_sales_order_key'))
    final_sales_df = unique_orders.join(sales_with_forext_df,unique_orders['unique_sales_order_key']==sales_with_forext_df['SALES_ORDER_KEY'],join_type='inner')
    final_sales_df = final_sales_df.select(
        col('SALES_ORDER_KEY'),
        col('ORDER_ID'),
        col('ORDER_DT'),
        col('CUSTOMER_NAME'),
        col('MOBILE_KEY'),
        col('Country'),
        col('Region'),
        col('ORDER_QUANTITY'),
        lit('EUR').alias('LOCAL_CURRENCY'),
        col('UNIT_PRICE').alias('LOCAL_UNIT_PRICE'),
        col('PROMOTION_CODE').alias('PROMOTION_CODE'),
        col('FINAL_ORDER_AMOUNT').alias('LOCAL_TOTAL_ORDER_AMT'),
        col('TAX_AMOUNT').alias('local_tax_amt'),
        col('USD2FR').alias("Exhchange_Rate"),
        (col('FINAL_ORDER_AMOUNT')/col('USD2FR')).alias('US_TOTAL_ORDER_AMT'),
        (col('TAX_AMOUNT')/col('USD2FR')).alias('USD_TAX_AMT'),
        col('payment_status'),
        col('shipping_status'),
        col('payment_method'),
        col('payment_provider'),
        col('phone').alias('conctact_no'),
        col('shipping_address')
    )

    final_sales_df.write.save_as_table("sales_dwh.curated.fr_sales_order",mode="append")
    
if __name__ == '__main__':
    main()

02:52:29 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.10.2, Platform: Windows-10-10.0.22621-SP0
02:52:29 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


02:52:30 - INFO - Snowpark Session information: 
"version" : 1.5.1,
"python.version" : 3.10.2,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1418670647832614,
"os.name" : Windows

02:52:30 - INFO - query: [select * from sales_dwh.source.in_sales_order]
02:52:30 - INFO - query execution done
02:52:30 - INFO - Number of results in first chunk: 0
02:52:30 - INFO - query: [select * from sales_dwh.common.exchange_rate]
02:52:30 - INFO - query execution done
02:52:30 - INFO - Number of results in first chunk: 0
02:52:30 - INFO - query: [SELECT "SALES_ORDER_KEY" AS "SALES_ORDER_KEY", "ORDER_ID" AS "ORDER_ID", "CUSTOM...]
02:52:31 - INFO - query execution done
02:52:31 - INFO - Number of results in first chunk: 0
02:52:31 - INFO - query: [SELECT "DATE" AS "DATE", "USD2USD" AS "USD2USD", "USD2EU" AS "USD2EU", "USD2CAN"...]
02:52:31 - INFO - query execution done
02:52:31 - INFO - Number of results in first chunk: 0
02:52:31 - INFO - query: [SELECT  *  FROM (( SELECT NULL ::

In [11]:
import sys
import logging
import pandas as pd

from snowflake.snowpark import Session, DataFrame, CaseExpr
from snowflake.snowpark.functions import col,lit,row_number, rank, split,cast, when, expr,min, max
from snowflake.snowpark.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType
from snowflake.snowpark import Window

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"ek87911.ap-southeast-1",
        "USER":"snowpark_user",
        "PASSWORD":"Test@12$4",
        "ROLE":"SYSADMIN",
        "DATABASE":"SNOWFLAKE_SAMPLE_DATA",
        "SCHEMA":"TPCH_SF1",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    return Session.builder.configs(connection_parameters).create()   

# This is a simple dim table having nation and region.
# fields are 'Country','Region'
def create_region_dim(all_sales_df,session)-> None:
    region_dim_df = all_sales_df.groupBy(col("Country"),col("Region")).count()
    region_dim_df.show(2)
    region_dim_df = region_dim_df.with_column("isActive",lit('Y'))
    region_dim_df = region_dim_df.selectExpr("sales_dwh.source.region_dim_seq.nextval as region_id_pk","Country", "Region", "isActive") 
    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    region_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_region_dim_df = session.sql("select Country, Region from sales_dwh.consumption.region_dim")

    region_dim_df = region_dim_df.join(existing_region_dim_df,region_dim_df['Country']==existing_region_dim_df['Country'],join_type='leftanti')
    region_dim_df.show(5)
    intsert_cnt = int(region_dim_df.count())
    if intsert_cnt>0:
        region_dim_df.write.save_as_table("sales_dwh.consumption.region_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")


    # have exclude key


def create_product_dim(all_sales_df,session)-> None:

    product_dim_df = all_sales_df.with_column("Brand",split(col('MOBILE_KEY'),lit('/'))[0])     \
                                .with_column("Model",split(col('MOBILE_KEY'),lit('/'))[1])      \
                                .with_column("Color",split(col('MOBILE_KEY'),lit('/'))[2])      \
                                .with_column("Memory",split(col('MOBILE_KEY'),lit('/'))[3])     \
                                .select(col('mobile_key'),col('Brand'),col('Model'),col('Color'),col('Memory'))
    
    product_dim_df = product_dim_df.select(col('mobile_key'),                    \
                                        cast(col('Brand'), StringType()).as_("Brand"),\
                                        cast(col('Model'), StringType()).as_("Model"),\
                                        cast(col('Color'), StringType()).as_("Color"),\
                                        cast(col('Memory'), StringType()).as_("Memory")\
                                        )
    
    product_dim_df = product_dim_df.groupBy(col('mobile_key'),col("Brand"),col("Model"),col("Color"),col("Memory")).count()
    product_dim_df = product_dim_df.with_column("isActive",lit('Y'))

    #fetch existing product dim records.
    existing_product_dim_df = session.sql("select mobile_key, Brand, Model, Color, Memory from sales_dwh.consumption.product_dim")
    existing_product_dim_df.count()

    product_dim_df = product_dim_df.join(existing_product_dim_df,["mobile_key", "Brand", "Model", "Color", "Memory"],join_type='leftanti')

    product_dim_df.show(5)

    product_dim_df = product_dim_df.selectExpr("sales_dwh.consumption.product_dim_seq.nextval as product_id_pk","mobile_key","Brand", "Model","Color","Memory", "isActive") 

    product_dim_df.show(5)
    intsert_cnt = int(product_dim_df.count())
    if intsert_cnt>0:
        product_dim_df.write.save_as_table("sales_dwh.consumption.product_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def create_promocode_dim(all_sales_df,session)-> None:


    promo_code_dim_df = all_sales_df.with_column( "promotion_code", expr("case when promotion_code is null then 'NA' else promotion_code end"))
    promo_code_dim_df = promo_code_dim_df.groupBy(col("promotion_code"),col("country"),col("region")).count()
    promo_code_dim_df = promo_code_dim_df.with_column("isActive",lit('Y'))

    #promo_code_dim_df.show(10)

    
    #fetch existing product dim records.
    existing_promo_code_dim_df = session.sql("select promotion_code, country, region from sales_dwh.consumption.promo_code_dim")


    promo_code_dim_df = promo_code_dim_df.join(existing_promo_code_dim_df,["promotion_code", "country", "region"],join_type='leftanti')


    promo_code_dim_df = promo_code_dim_df.selectExpr("sales_dwh.consumption.promo_code_dim_seq.nextval as promo_code_id_pk","promotion_code", "country","region","isActive") 


    intsert_cnt = int(promo_code_dim_df.count())
    if intsert_cnt>0:
        promo_code_dim_df.write.save_as_table("sales_dwh.consumption.promo_code_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")
    
def create_customer_dim(all_sales_df, session) -> None:
    customer_dim_df = all_sales_df.groupBy(col("COUNTRY"),col("REGION"),col("CUSTOMER_NAME"),col("CONCTACT_NO"),col("SHIPPING_ADDRESS")).count()
    customer_dim_df = customer_dim_df.with_column("isActive",lit('Y'))
    customer_dim_df = customer_dim_df.selectExpr("customer_name", "conctact_no","shipping_address","country","region" ,"isactive") 
    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    customer_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_customer_dim_df = session.sql("select customer_name,conctact_no,shipping_address,country, region from sales_dwh.consumption.customer_dim")

    customer_dim_df = customer_dim_df.join(existing_customer_dim_df,["customer_name","conctact_no","shipping_address","country", "region"],join_type='leftanti')

    customer_dim_df = customer_dim_df.selectExpr("sales_dwh.consumption.customer_dim_seq.nextval as customer_id_pk","customer_name", "conctact_no","shipping_address","country","region", "isActive") 

    customer_dim_df.show(5)

    intsert_cnt = int(customer_dim_df.count())
    if intsert_cnt>0:
        customer_dim_df.write.save_as_table("sales_dwh.consumption.customer_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")
    
def create_payment_dim(all_sales_df, session) -> None:
    payment_dim_df = all_sales_df.groupBy(col("COUNTRY"),col("REGION"),col("payment_method"),col("payment_provider")).count()
    payment_dim_df = payment_dim_df.with_column("isActive",lit('Y'))

    #region_dim_df.write.save_as_table('sales_dwh.consumption.region_dim',mode="append")   

    payment_dim_df.show(5)
    # part 2 where delta data will be processed 
    
    existing_payment_dim_df = session.sql("select payment_method,payment_provider,country, region from sales_dwh.consumption.payment_dim")

    payment_dim_df = payment_dim_df.join(existing_payment_dim_df,["payment_method","payment_provider","country", "region"],join_type='leftanti')

    payment_dim_df = payment_dim_df.selectExpr("sales_dwh.consumption.payment_dim_seq.nextval as payment_id_pk","payment_method", "payment_provider","country","region", "isActive") 


    intsert_cnt = int(payment_dim_df.count())
    if intsert_cnt>0:
        payment_dim_df.write.save_as_table("sales_dwh.consumption.payment_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def create_date_dim(all_sales_df, session) -> None:
    start_date = all_sales_df.select(min("order_dt").alias("min_order_dt")).collect()[0].as_dict()['MIN_ORDER_DT']
    end_date = all_sales_df.select(max("order_dt").alias("max_order_dt")).collect()[0].as_dict()['MAX_ORDER_DT']
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    #print(date_range)
    date_dim = pd.DataFrame()
    date_dim['order_dt'] = date_range.date
    date_dim['Year'] = date_range.year
    # Calculate day counter
    start_day_of_year = pd.to_datetime(start_date).dayofyear
    date_dim['DayCounter'] = date_range.dayofyear - start_day_of_year + 1

    date_dim['Month'] = date_range.month
    date_dim['Quarter'] = date_range.quarter
    date_dim['Day'] = date_range.day
    date_dim['DayOfWeek'] = date_range.dayofweek
    date_dim['DayName'] = date_range.strftime('%A')
    date_dim['DayOfMonth'] = date_range.day
    date_dim['Weekday'] = date_dim['DayOfWeek'].map({0: 'Weekday', 1: 'Weekday', 2: 'Weekday', 3: 'Weekday', 4: 'Weekday', 5: 'Weekend', 6: 'Weekend'})


    date_dim_df = session.create_dataframe(date_dim)

    existing_date_dim_df = session.sql("select order_dt from sales_dwh.consumption.date_dim ") 
    date_dim_df = date_dim_df.join(existing_date_dim_df,existing_date_dim_df['order_dt']==date_dim_df['"order_dt"'],join_type='leftanti')

    date_dim_df = date_dim_df.selectExpr(' \
                                   sales_dwh.consumption.date_dim_seq.nextval, \
                                   "order_dt" as order_dt, \
                                   "DayCounter" as day_counter,\
                                   "Year" as order_year, \
                                   "Month" as order_month, \
                                   "Quarter" as order_quarter, \
                                   "Day" as order_day, \
                                   "DayOfWeek" as order_dayofweek, \
                                   "DayName" as order_dayname, \
                                   "DayOfMonth" as order_dayofmonth, \
                                   "Weekday" as order_weekday\
                                   ')

    intsert_cnt = int(date_dim_df.count())
    if intsert_cnt>0:
        date_dim_df.write.save_as_table("sales_dwh.consumption.date_dim",mode="append")
        print("save operation ran...")
    else:
        print("No insert ...Opps...")

def main():
    #get the session object and get dataframe
    session = get_snowpark_session()

    in_sales_df = session.sql("select * from sales_dwh.curated.in_sales_order")
    us_sales_df = session.sql("select * from sales_dwh.curated.us_sales_order")
    fr_sales_df = session.sql("select * from sales_dwh.curated.fr_sales_order")

    all_sales_df = in_sales_df

    create_date_dim(all_sales_df,session)       #date dimension


    create_region_dim(all_sales_df,session)     #region dimension
    create_product_dim(all_sales_df,session)    #product dimension
    create_promocode_dim(all_sales_df,session)  #promot code dimension
    create_customer_dim(all_sales_df,session)   #customer dimension
    create_payment_dim(all_sales_df,session)    #payment dimension
    create_date_dim(all_sales_df,session)       #date dimension


    date_dim_df = session.sql("select date_id_pk, order_dt from sales_dwh.consumption.date_dim")
    customer_dim_df = session.sql("select customer_id_pk, customer_name, country, region from sales_dwh.consumption.CUSTOMER_DIM")
    payment_dim_df = session.sql("select payment_id_pk, payment_method, payment_provider, country, region from sales_dwh.consumption.PAYMENT_DIM")
    product_dim_df = session.sql("select product_id_pk, mobile_key from sales_dwh.consumption.PRODUCT_DIM")
    promo_code_dim_df = session.sql("select promo_code_id_pk,promotion_code,country, region from sales_dwh.consumption.PROMO_CODE_DIM")
    region_dim_df = session.sql("select region_id_pk,country, region from sales_dwh.consumption.REGION_DIM")

    all_sales_df = all_sales_df.with_column( "promotion_code", expr("case when promotion_code is null then 'NA' else promotion_code end"))
    all_sales_df = all_sales_df.join(date_dim_df, ["order_dt"],join_type='inner')
    all_sales_df = all_sales_df.join(customer_dim_df, ["customer_name","region","country"],join_type='inner')
    all_sales_df = all_sales_df.join(payment_dim_df, ["payment_method", "payment_provider", "country", "region"],join_type='inner')
    #all_sales_df = all_sales_df.join(product_dim_df, ["brand","model","color","Memory"],join_type='inner')
    all_sales_df = all_sales_df.join(product_dim_df, ["mobile_key"],join_type='inner')
    all_sales_df = all_sales_df.join(promo_code_dim_df, ["promotion_code","country", "region"],join_type='inner')
    all_sales_df = all_sales_df.join(region_dim_df, ["country", "region"],join_type='inner')
    all_sales_df = all_sales_df.selectExpr("sales_dwh.consumption.sales_fact_seq.nextval as order_id_pk, \
                                           order_id as order_code,                               \
                                           date_id_pk as date_id_fk,          \
                                           region_id_pk as region_id_fk,            \
                                           customer_id_pk as customer_id_fk,        \
                                           payment_id_pk as payment_id_fk,          \
                                           product_id_pk as product_id_fk,          \
                                           promo_code_id_pk as promo_code_id_fk,    \
                                           order_quantity,                          \
                                           local_total_order_amt,                   \
                                           local_tax_amt,                           \
                                           exhchange_rate,                          \
                                           us_total_order_amt,                      \
                                           usd_tax_amt                              \
                                           ")
    all_sales_df.write.save_as_table("sales_dwh.consumption.sales_fact",mode="append")


if __name__ == '__main__':
    main()

02:52:38 - INFO - Snowflake Connector for Python Version: 3.0.4, Python Version: 3.10.2, Platform: Windows-10-10.0.22621-SP0
02:52:38 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
02:52:39 - INFO - Snowpark Session information: 
"version" : 1.5.1,
"python.version" : 3.10.2,
"python.connector.version" : 3.0.4,
"python.connector.session.id" : 1418670647836718,
"os.name" : Windows

02:52:39 - INFO - query: [select * from sales_dwh.curated.in_sales_order]
02:52:40 - INFO - query execution done
02:52:40 - INFO - Number of results in first chunk: 0
02:52:40 - INFO - query: [SELECT min("ORDER_DT") AS "MIN_ORDER_DT" FROM (select * from sales_dwh.curated.i...]
02:52:40 - INFO - query execution done
02:52:40 - INFO - Number of results in first chunk: 1
02:52:40 - INFO - query: [SELECT max("ORDER_D

MissingDependencyError: Missing optional dependency: pandas